# **Cleaned up code that works (creates logical masks of bright and dark matrices separately, then adds them together)**

In [ ]:
# 06/17/2024

import os
import numpy as np
import pandas as pd
from skimage import io, morphology, filters, measure
from skimage.morphology import remove_small_objects
from skimage.measure import regionprops
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.ndimage import label
from scipy.ndimage import center_of_mass, find_objects
from scipy.spatial.distance import pdist
import cv2
from PIL import Image
import math
from scipy.signal import wiener
import tifffile as tiff

def DICAnalysis(Path, ID, X, Y, Sframe, Eframe):
  # setting up variables
  stackPath, stackFile = os.path.split(Path)
  startColX = X
  startRowY = Y
  particleID = ID
  SFrame = Sframe
  EFrame = Eframe

  # processing paremetersok
  frameRange = 20 #used in moving average
  threshStack = 10 #threshold for indentifying none-bkg pixels
  threshFrame = 0.6 #threshole for indentifying none-bkw pixeld within ROI
  particleSizeThresh = 20 #number of pixels in particle
  jumpThresh = 20 #longest distance allowen in adjacent thresh for NP tracking
  ROI = 50 #ROI around particle
  ROISmall = 18 # region around particle
  particleDefaultDiameter = 10
  minArea = np.floor(np.pi*(particleDefaultDiameter/2)**2) #pixels

  analyzeMode = "track"

  #camera parameters
  exposureTime = 0.100
  pixelSize = 0.065

  #set up for saving
  saveID = os.path.splitext(stackFile)[0]
  savingPath = f'{stackPath}/Process/Particle {particleID}/'
  if not os.path.isdir(savingPath):
    os.makedirs(savingPath)
  saveBaseName = f'{savingPath}{saveID}_Particle{particleID}_frameRange{frameRange}_X{startColX}_Y{startRowY}'
  saveBaseNameMovie = f'{savingPath}{saveID}_frameRange{frameRange}'

  # read image
  stackName = os.path.join(stackPath, stackFile)
  stackFrames = EFrame - SFrame + 1
  stack = io.imread(stackName)
  stackFrames = stack.shape[0]
  height, width = stack.shape[1], stack.shape[2]

  if not math.isfinite(EFrame):
    EFrame = stackFrames

  stackMatrix = np.zeros((stackFrames, height, width), dtype=np.uint16)
  for i in range(stackFrames):
    stackMatrix[i, :, :] = stack[i + SFrame - 1]

  #pre allocate matrices
  subtractedMatrixDouble = np.zeros_like(stackMatrix, dtype=np.float64)  # values will be doubles
  subtractedMatrix = np.zeros_like(stackMatrix, dtype=np.int16)  # values will be unsigned integers
  subtractedMatrixB = np.zeros_like(stackMatrix, dtype=np.int16)
  subtractedMatrixD = np.zeros_like(stackMatrix, dtype=np.int16)
  for i in range(stackFrames):
    # calculate moving average
    startFrame = max(i - frameRange // 2, 1)
    endFrame = min(startFrame + frameRange, stackFrames)
    if (endFrame - startFrame) < (frameRange - 1):
      startFrame = endFrame - frameRange
    averageImage = np.mean(stackMatrix[startFrame:endFrame, :, :], axis=0).astype(np.uint16) #mean along 3rd dimention (depth/time)

    # Calculate subtracted image as a double so negative values do not get deleted; aka preserves bright/dark contrast over middle background
    imageMatrixDouble = stackMatrix[i, :, :].astype(np.float64)
    subtractedMatrixDouble[i, :, :] = imageMatrixDouble - averageImage.astype(np.float64)
    subtractedMatrix[i, :, :] = np.abs(subtractedMatrixDouble[i, :, :]).astype(np.uint16)
    subtractedMatrixB[i, :, :] = (stackMatrix[i, :, :] - (averageImage)).astype(np.uint16)
    subtractedMatrixD[i, :, :] = (averageImage - stackMatrix[i, :, :]).astype(np.uint16)
    subtractedMatrix[i, :, :] = np.add(abs(subtractedMatrixB[i, :, :]), abs(subtractedMatrixD[i, :, :]))  # this movie will be bright where there are particles and dark everywhere else, for localization purposes, not intensity.

  minVal= np.abs(np.min(np.min(np.min(subtractedMatrixDouble)))) #get the darkest pixel in all frames
  subtractedMatrixDoubleSaving = (subtractedMatrixDouble + minVal + 100).astype(np.uint16) #make the lowest pixel intensity as 100

  # auto tracking
  # preallocate matrices
  particleRegionROI = np.zeros((stackFrames, 2*ROI + 1, 2*ROI+ 1), dtype=bool)
  particleRegionD = np.zeros_like(subtractedMatrix, dtype=bool)  # label the possible particles (including vesicles) within the whole image
  particleRegionB = np.zeros_like(subtractedMatrix, dtype=bool)  # label the possible particles (including vesicles) within the whole image
  particleRegion = np.zeros_like(subtractedMatrix, dtype=bool)  # label the possible particles (including vesicles) within the whole image
  particleCentroidROI = np.zeros((stackFrames, 2))
  previousCoord = [ROI+1, ROI+1]

  fig = plt.figure() #what is this
  fig.set_size_inches(10.5, 10.5, forward=True)
  fig.tight_layout(pad=3.0)
  #show
  manager = plt.get_current_fig_manager()

  JumpFrames = 0
  for j in range(stackFrames):
    # determine none-bkg pixels from uint16 matrix (all positive int. of
    # NP) under the assumption that most pixels are bkg
    averageUint=np.mean(np.mean(subtractedMatrix[j,:,:])) #average of the full frame subtracted matrix (uint16)
    stdevUint=np.std(np.std(subtractedMatrix[j,:,:].astype(np.float64)))

    # Create logical mask by filtering by intensity and size

    particleRegionD[j,:,:] = subtractedMatrixD[j, :, :] > (averageUint + (threshStack * stdevUint))
    particleRegionD[j,:,:] = remove_small_objects(particleRegionD[j,:,:], min_size=particleSizeThresh)

    particleRegionB[j,:,:] = subtractedMatrixB[j, :, :] > (averageUint + (threshStack * stdevUint))
    particleRegionB[j,:,:] = remove_small_objects(particleRegionB[j,:,:], min_size=particleSizeThresh)

    particleRegion[j, :, :] = particleRegionD[j,:,:]+particleRegionB[j,:,:]

    # extract particle region within the ROI
    particleRegionROI[j, :, :] = particleRegion[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1]
    # label regions in the logical mask
    L, num_features = label(particleRegionROI[j, :, :])
    # get properties of the possible particles regions (the centroid and bounding box)
    slices = find_objects(L)
    # calculate centroids and bounding boxes
    centroids = [center_of_mass(particleRegionROI[j, :, :], L, i + 1) for i in range(num_features)]
    bounding_boxes = [sli for sli in slices]
    # combine centroids and bounding boxes into particleProps
    particleProps = [{'Centroid': centroids[i], 'BoundingBox': bounding_boxes[i]} for i in range(num_features)]

    if j>1:
      previousCoord = particleCentroidROI[j-1, :]
    if j<10: # plots logical mask for checking code
      plt.figure(figsize=(6, 6))
      plt.imshow(subtractedMatrix[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1], cmap='gray')
      plt.colorbar(label='Value')
      plt.title('Subtracted Matrix ROI')
      plt.show()

      plt.figure(figsize=(6, 6))
      plt.imshow(particleRegionB[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1], cmap='gray')
      plt.colorbar(label='Value')
      plt.title('Boolean Matrix B ROI')
      plt.show()

      plt.figure(figsize=(6, 6))
      plt.imshow(particleRegionD[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1], cmap='gray')
      plt.colorbar(label='Value')
      plt.title('Boolean Matrix D ROI')
      plt.show()

      print(f"particleCentroidROI[j-1, :]: {particleCentroidROI[j-1, :]}")
      plt.figure(figsize=(6, 6))
      plt.imshow(particleRegionROI[j, :, :], cmap='gray')
      plt.colorbar(label='Value')
      plt.title('Boolean Matrix Plot')
      plt.show()

    # if more than one region identified, choose the one closest to
    # previous centroid (for confined and regular diffusion). For active
    # transport, this can be a problem
    if len(particleProps) >= 2:
      coordList = np.zeros((len(particleProps), 2))
      distances = np.zeros(len(particleProps))
      for h in range(len(particleProps)):
        # get the centroid coordinates
        coordList[h, :] = particleProps[h]['Centroid']
        # compute the distance between previousCoord and the current centroid
        distances[h] = pdist([previousCoord, coordList[h, :]], 'euclidean')[0]
        # update chosenParticle with the index of the minimum distance
        chosenParticle = np.argmin(distances[:h+1]) # try removing +1
      particleProps = [particleProps[chosenParticle]]
      particleRegionROI[j, :, :] = (L == (chosenParticle + 1))  # +1 because Python's label function starts from 1

    if len(particleProps) == 1:
      centroid = particleProps[0]['Centroid']
      particleCentroidROI[j, :] = centroid
      # if the particle jumped too far, set the centroid as last frame
      # location and define a 'default particle mask' centered around the
      # centroid.
      distanceFromPrev = pdist(np.array([previousCoord, particleCentroidROI[j, :]]))[0]
      if distanceFromPrev > jumpThresh:
        JumpFrames += 1
        particleCentroidROI[j, :] = previousCoord  # assume particle didn't move from last frame

        # create a meshgrid for the ROI
        rr, cc = np.meshgrid(np.arange(1, 2*ROI+2), np.arange(1, 2*ROI+2))  # Adjust for Python's 0-indexing

        # update particleRegionROI to include the current mask
        particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2

    if len(particleProps) == 0:
      JumpFrames += 1
      particleCentroidROI[j, :] = previousCoord  # assume particle didn't move from last frame
      rr, cc = np.meshgrid(np.arange(1, 2*ROI + 2), np.arange(1, 2*ROI + 2))
      particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2

    particlePropsFinal = regionprops(particleRegionROI[j, :, :].astype(int))
    if particlePropsFinal and particlePropsFinal[0].area < minArea:
      # create a meshgrid for the ROI
      rr, cc = np.meshgrid(np.arange(1, 2*ROI+2), np.arange(1, 2*ROI+2))

      # Update particleRegionROI to include the current mask
      particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2

  centroidGlobalColX = np.round(particleCentroidROI[:, 1]) + (startColX - (ROI + 1))
  centroidGlobalRowY = np.round(particleCentroidROI[:, 0]) + (startRowY - (ROI + 1))

  T = pd.DataFrame({
      'centroidGlobalColX': centroidGlobalColX,
      'centroidGlobalRowY': centroidGlobalRowY
  })

  # print the DataFrame to verify
  print(T)

  # save the DataFrame to an xlsx file
  T.to_excel(f'{saveBaseName}_Coords.xlsx', index=False)

  # calculation

  rawROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=np.uint16)
  nonebkgROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=bool)
  bkgregionROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=bool)
  localAverage = np.zeros(stackFrames)
  localStdev = np.zeros(stackFrames)
  particleSignal = np.zeros(stackFrames)
  particleSignalBright = np.zeros(stackFrames)
  particleSignalDark = np.zeros(stackFrames)
  nanCountBrights = np.zeros(stackFrames)
  nanCountDarks = np.zeros(stackFrames)
  numberBrightPixels = np.zeros(stackFrames)
  numberDarkPixels = np.zeros(stackFrames)

  for j in range(stackFrames):
    rawROI[j, :, :] = stack[j, startRowY - ROI:startRowY + ROI + 1, startColX - ROI:startColX + ROI + 1]
    rawROIHolder = rawROI[j, :, :]
    averageROI = np.mean(rawROIHolder)
    NPidentifyROI = filters.median(rawROIHolder - averageROI) + filters.median(averageROI - rawROIHolder)
    averageFrame = np.mean(NPidentifyROI)
    stdevFrame = np.std(NPidentifyROI)
    nonebkgROI[j, ...] = morphology.remove_small_objects(NPidentifyROI > (averageFrame + threshFrame * stdevFrame), particleSizeThresh)
    bkgregionROI[j, ...] = ~nonebkgROI[j, ...] & ~particleRegionROI[j, ...]

    particleCentroidROIround = np.round(particleCentroidROI[j, :]).astype(int)

    if (particleCentroidROIround[0] - ROISmall < 0 or particleCentroidROIround[1] - ROISmall < 0 or
            particleCentroidROIround[0] + ROISmall >= 2 * ROI + 1 or particleCentroidROIround[1] + ROISmall >= 2 * ROI + 1):
        print('Index Error')
        continue

    rawROIholderLocal = rawROI[
        j,
        particleCentroidROIround[1] - ROISmall:particleCentroidROIround[1] + ROISmall + 1,
        particleCentroidROIround[0] - ROISmall:particleCentroidROIround[0] + ROISmall + 1
    ]
    notParticlesLocal = bkgregionROI[
        j,
        particleCentroidROIround[1] - ROISmall:particleCentroidROIround[1] + ROISmall + 1,
        particleCentroidROIround[0] - ROISmall:particleCentroidROIround[0] + ROISmall + 1
    ]

    if np.any(notParticlesLocal):  # check if there are any true elements
      localAverage[j] = np.mean(rawROIholderLocal[notParticlesLocal])
      localStdev[j] = np.std(rawROIholderLocal[notParticlesLocal])
    else:
      localAverage[j] = 0
      localStdev[j] = 0

    particleSignal[j] = np.mean(rawROIHolder[particleRegionROI[j, ...]])

    particleSignalList = rawROIHolder[particleRegionROI[j, ...]]
    particleSignalListBright = particleSignalList > localAverage[j]
    particleSignalListDark = particleSignalList < localAverage[j]
    numberBrightPixels[j] = np.sum(particleSignalListBright)
    numberDarkPixels[j] = np.sum(particleSignalListDark)

    if np.any(particleSignalListBright):  # check if there are any true elements
        particleSignalBright[j] = np.mean(particleSignalList[particleSignalListBright])
    else:
        particleSignalBright[j] = localAverage[j]
        nanCountBrights[j] = 1

    if np.any(particleSignalListDark):  # check if there are any true elements
        particleSignalDark[j] = np.mean(particleSignalList[particleSignalListDark])
    else:
        particleSignalDark[j] = localAverage[j]
        nanCountDarks[j] = 1

  brightContrast = (particleSignalBright - localAverage) / localAverage
  darkContrast = (particleSignalDark - localAverage) / localAverage
  brightContrastNorm = (brightContrast - np.min(brightContrast)) / (np.max(brightContrast) - np.min(brightContrast))
  darkContrastNorm = (darkContrast - np.max(darkContrast)) / (np.min(darkContrast) - np.max(darkContrast))
  contrastDifference = brightContrastNorm - darkContrastNorm
  contrast = (particleSignal - localAverage) / localAverage

  particleCentroidROIColXSmooth = gaussian_filter(particleCentroidROI[:, 1], sigma=2)
  particleCentroidROIRowYSmooth = gaussian_filter(particleCentroidROI[:, 0], sigma=2)
  centroidGlobalColXSmooth = gaussian_filter(centroidGlobalColX, sigma=2)
  centroidGlobalRowYSmooth = gaussian_filter(centroidGlobalRowY, sigma=2)

  positionColX_um = centroidGlobalColX * pixelSize
  positionRowY_um = centroidGlobalRowY * pixelSize
  positionColXFirst = positionColX_um[0]
  positionRowYFirst = positionRowY_um[0]
  displacement_um = np.sqrt((positionColX_um - positionColXFirst) ** 2 + (positionRowY_um - positionRowYFirst) ** 2)
  displacement_umSmooth = gaussian_filter(displacement_um, sigma=2)

  # saving results
  frameNum = np.arange(1, stackFrames + 1)
  timeStampExp = np.arange(0, stackFrames * exposureTime, exposureTime)

  result_df = pd.DataFrame({
    'frameNum': frameNum, 'timeStampExp': timeStampExp, 'localAverage': localAverage,
    'localStdev': localStdev, 'particleSignal': particleSignal, 'contrast': contrast,
    'particleSignalBright': particleSignalBright, 'particleSignalDark': particleSignalDark,
    'brightContrast': brightContrast, 'darkContrast': darkContrast, 'brightContrastNorm': brightContrastNorm,
    'darkContrastNorm': darkContrastNorm, 'contrastDifference': contrastDifference,
    'particleCentroidROIX': particleCentroidROI[:, 1], 'particleCentroidROIY': particleCentroidROI[:, 0],
    'particleCentroidROIColXSmooth': particleCentroidROIColXSmooth, 'particleCentroidROIRowYSmooth': particleCentroidROIRowYSmooth,
    'centroidGlobalColX': centroidGlobalColX, 'centroidGlobalRowY': centroidGlobalRowY,
    'centroidGlobalColXSmooth': centroidGlobalColXSmooth, 'centroidGlobalRowYSmooth': centroidGlobalRowYSmooth,
    'displacement_um': displacement_um, 'displacement_umSmooth': displacement_umSmooth
  })
  result_df.to_excel(f'{saveBaseName}_Result.xlsx', index=False)

  parameters_df = pd.DataFrame({
    'imageFolder': [stackPath], 'imageFile': [stackFile], 'SFrame': [SFrame], 'EFrame': [EFrame],
    'startColX': [startColX], 'startRowY': [startRowY], 'analyzeMode': [analyzeMode], 'ROI': [ROI],
    'ROISmall': [ROISmall], 'frameRange': [frameRange], 'threshStack': [threshStack], 'threshFrame': [threshFrame],
    'particleSizeThresh': [particleSizeThresh], 'particleDefaultDiameter': [particleDefaultDiameter],
    'minArea': [minArea], 'jumpThresh': [jumpThresh], 'exposureTime': [exposureTime], 'pixelSize': [pixelSize]
  })
  parameters_df.to_excel(f'{saveBaseName}_Parameters.xlsx', index=False)

  rawROI_uint16 = rawROI.astype(np.uint16)
  io.imsave(f'{saveBaseName}_ROI.tif', rawROI_uint16)

  particleidentified = np.zeros_like(rawROI_uint16)
  for h in range(stackFrames):
    roiframe = rawROI[h, ...]
    particleframe = np.zeros_like(roiframe)
    particleframe[particleRegionROI[h, ...]] = roiframe[particleRegionROI[h, ...]]
    particleframe[~particleRegionROI[h, ...]] = localAverage[h] - 100
    particleidentified[h, ...] = particleframe

  io.imsave(f'{saveBaseNameMovie}_Particle {particleID}.tif', particleidentified)


  # plots
  plt.figure()
  plt.plot(timeStampExp, localAverage, label='Local background')
  plt.plot(timeStampExp, localAverage + localStdev, label='+ Local stdev')
  plt.plot(timeStampExp, localAverage - localStdev, label='- Local stdev')
  plt.plot(timeStampExp, particleSignal, label='Particle signal')
  plt.title(f'Mean Signal, file {saveID}, Particle {particleID}, StartX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Intensity')
  plt.savefig(f'{saveBaseName}_SignalMean.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, localAverage, label='Local background')
  plt.plot(timeStampExp, localAverage + localStdev, label='+ Local stdev')
  plt.plot(timeStampExp, localAverage - localStdev, label='- Local stdev')
  plt.plot(timeStampExp, particleSignalBright, label='Bright signal')
  plt.plot(timeStampExp, particleSignalDark, label='Dark signal')
  plt.title(f'Bright and Dark Signal, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Intensity')
  plt.savefig(f'{saveBaseName}_SignalBrightDark.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, brightContrast, label='Bright Contrast')
  plt.plot(timeStampExp, darkContrast, label='Dark Contrast')
  plt.title(f'Bright and Dark Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast')
  plt.savefig(f'{saveBaseName}_BD_Contrast.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, brightContrastNorm, label='Normalized Bright Contrast')
  plt.plot(timeStampExp, darkContrastNorm, label='Normalized Dark Contrast')
  plt.title(f'Normalized Bright and Dark Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Normalized Contrast')
  plt.savefig(f'{saveBaseName}_ContrastNorm.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, contrastDifference)
  plt.plot(timeStampExp, np.zeros(stackFrames), label='Zero line')
  plt.title(f'Contrast Difference, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast Difference')
  plt.ylim([-1, 1])
  plt.savefig(f'{saveBaseName}_ContrastDiff.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, contrast)
  plt.plot(timeStampExp, np.zeros(stackFrames), label='Zero line')
  plt.title(f'Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast')
  plt.savefig(f'{saveBaseName}_Contrast.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, centroidGlobalColX, label='X position')
  plt.plot(timeStampExp, centroidGlobalColXSmooth, label='X position smooth')
  plt.title(f'X position, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('X position (pixels)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_positionX.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, centroidGlobalRowY, label='Y position')
  plt.plot(timeStampExp, centroidGlobalRowYSmooth, label='Y position smooth')
  plt.title(f'Y position, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Y position (pixels)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_positionY.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, displacement_um, label='Displacement')
  plt.plot(timeStampExp, displacement_umSmooth, label='Displacement smooth')
  plt.title(f'Displacement (um), file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Displacement (um)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_Displacement.eps')
  plt.show()

  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.scatter(centroidGlobalColX[0], centroidGlobalRowY[0], color='red')
  plt.plot(centroidGlobalColX, centroidGlobalRowY, color='yellow')
  plt.title(f'Overlay, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_Overlay.eps')
  plt.show()

  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.scatter(centroidGlobalColX[0], centroidGlobalRowY[0], color='red')
  plt.plot(centroidGlobalColXSmooth, centroidGlobalRowYSmooth, color='yellow')
  plt.title(f'Overlay Smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlaySmooth.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.plot(particleCentroidROI[:, 1], particleCentroidROI[:, 0], color='yellow')
  plt.title(f'Overlay zoom, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlayZoom.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.plot(particleCentroidROIColXSmooth, particleCentroidROIRowYSmooth, color='yellow')
  plt.title(f'Overlay zoom smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlayZoomSmooth.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(centroidGlobalColX))
  plt.scatter(centroidGlobalColX, centroidGlobalRowY, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(centroidGlobalRowY[0], centroidGlobalColX[0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'Full path, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathFull.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(particleCentroidROI))
  plt.scatter(particleCentroidROI[:, 1], particleCentroidROI[:, 0], c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'ROI path, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathROI.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(centroidGlobalColXSmooth))
  plt.scatter(centroidGlobalColXSmooth, centroidGlobalRowYSmooth, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(centroidGlobalRowY[0], centroidGlobalColX[0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'Full path smooth, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathFullSmooth.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(particleCentroidROIColXSmooth))
  plt.scatter(particleCentroidROIColXSmooth, particleCentroidROIRowYSmooth, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'ROI path smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathROISmooth.eps')
  plt.show()

  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.title(f'Frame {SFrame}, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.savefig(f'{saveBaseName}_Frame{SFrame}.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.title(f'Raw ROI, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.savefig(f'{saveBaseName}_Frame1RawROI.eps')
  plt.show()

  plt.close('all')

# call the function
DICAnalysis('/content/drive/MyDrive/Colab Notebooks/Copy of Cell1_S1.tif', 'P1_C1_S1_11', 685, 660, 1, 300)



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Copy of Cell1_S1.tif'

# **Cleaned up code that removes negative values, then adds bright and dark matrices.**

It does not work yet, but I am working on it 06/17

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, morphology, filters, measure
from skimage.morphology import remove_small_objects
from skimage.measure import regionprops
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.ndimage import label
from scipy.ndimage import center_of_mass, find_objects
from scipy.spatial.distance import pdist
import cv2
from PIL import Image
import math
from scipy.signal import wiener
import tifffile as tiff


def DICAnalysis(Path, ID, X, Y, Sframe, Eframe):
  stackPath, stackFile = os.path.split(Path)

  startColX = X
  startRowY = Y
  particleID = ID
  SFrame = Sframe
  EFrame = Eframe

  analyzeMode = "track"

  # processing paremetersok
  frameRange = 20 #used in moving average
  threshStack = 10 #threshold for indentifying none-bkg pixels
  threshFrame = 0.6 #threshole for indentifying none-bkw pixeld within ROI
  particleSizeThresh = 20 #number of pixels in particle
  jumpThresh = 20 #longest distance allowen in adjacent thresh for NP tracking
  ROI = 50 #ROI around particle
  ROISmall = 18 # region around particle
  particleDefaultDiameter = 10
  minArea = np.floor(np.pi*(particleDefaultDiameter/2)**2) #pixels

  #camera parameters
  exposureTime = 0.100
  pixelSize = 0.065

  #set up for saving
  saveID = os.path.splitext(stackFile)[0]
  savingPath = f'{stackPath}/Process/Particle {particleID}/'
  if not os.path.isdir(savingPath):
    os.makedirs(savingPath)
  saveBaseName = f'{savingPath}{saveID}_Particle{particleID}_frameRange{frameRange}_X{startColX}_Y{startRowY}'
  saveBaseNameMovie = f'{savingPath}{saveID}_frameRange{frameRange}'
  print(f"saveID: {saveID}")
  print(f"savingPath: {savingPath}")
  print(f"saveBaseName: {saveBaseName}")
  print(f"saveBaseNameMovie: {saveBaseNameMovie}")

  # read image
  stackName = os.path.join(stackPath, stackFile)

  stackFrames = EFrame-SFrame+1
  stack = io.imread(stackName)

  # Get the number of frames and image dimensions
  stackFrames = stack.shape[0]
  height, width = stack.shape[1], stack.shape[2]

  print(f"Initial stackFrames: {stackFrames}")
  print(f"height: {height}, width: {width}")

  # Adjust the number of frames if EFrame is not finite
  if not math.isfinite(EFrame):
    EFrame = stackFrames

  #stackFrames = EFrame - SFrame + 1
  print(f"stackFrames: {stackFrames}")

  # Initialize stackMatrix
  stackMatrix = np.zeros((stackFrames, height, width), dtype=np.int16)
  for i in range(stackFrames):
    stackMatrix[i, :, :] = stack[i + SFrame - 1]

  #pre allocate matrices
  subtractedMatrixDouble = np.zeros_like(stackMatrix, dtype=np.float64)  # values will be doubles
  subtractedMatrix = np.zeros_like(stackMatrix, dtype=np.int16)  # values will be unsigned integers
  subtractedMatrixB = np.zeros_like(stackMatrix, dtype=np.int16)
  subtractedMatrixD = np.zeros_like(stackMatrix, dtype=np.int16)
  for i in range(stackFrames):
    #calculate moving average
    startFrame = max(i - frameRange // 2, 1)
    endFrame = min(startFrame + frameRange, stackFrames)
    if (endFrame - startFrame) < (frameRange - 1):
      startFrame = endFrame - frameRange
    averageImage = np.mean(stackMatrix[startFrame:endFrame, :, :], axis=0).astype(np.uint16) #mean along 3rd dimention (depth/time)
    #Calculate subtracted image as a double so negative values do not get deleted; aka preserves bright/dark contrast over middle background
    imageMatrixDouble = stackMatrix[i, :, :].astype(np.float64)
    subtractedMatrixDouble[i, :, :] = imageMatrixDouble - averageImage.astype(np.float64)
    subtractedMatrix[i, :, :] = np.abs(subtractedMatrixDouble[i, :, :]).astype(np.int16)

    subtractedMatrixB[i, :, :] = (stackMatrix[i, :, :] - (averageImage))
    subtractedMatrixD[i, :, :] = (averageImage - stackMatrix[i, :, :])
    subtractedMatrixB[subtractedMatrixB < 0] = 0
    subtractedMatrixD[subtractedMatrixD < 0] = 0
    subtractedMatrix[i, :, :] = np.add(subtractedMatrixB[i, :, :], subtractedMatrixD[i, :, :])  # this movie will be bright where there are particles and dark everywhere else, for localization purposes, not intensity.

  minVal= np.abs(np.min(np.min(np.min(subtractedMatrixDouble)))) #get the darkest pixel in all frames
  subtractedMatrixDoubleSaving = (subtractedMatrixDouble + minVal + 100).astype(np.int16) #make the lowest pixel intensity as 100

  subMatrixZerosTest = np.zeros_like(subtractedMatrix)
  minVal= np.abs(np.min(np.min(np.min(subtractedMatrixDouble)))) #get the darkest pixel in all frames
  subtractedMatrixDoubleSaving = (subtractedMatrixDouble + minVal + 100).astype(np.int16) #make the lowest pixel intensity as 100

  # auto tracking
  # preallocate matrices
  particleRegionROI = np.zeros((stackFrames, 2*ROI + 1, 2*ROI+ 1), dtype=bool)
  particleRegion = np.zeros_like(subtractedMatrix, dtype=bool)  # label the possible particles (including vesicles) within the whole image
  particleCentroidROI = np.zeros((stackFrames, 2))
  previousCoord = [ROI+1, ROI+1]
  fig = plt.figure()
  fig.set_size_inches(10.5, 10.5, forward=True)
  fig.tight_layout(pad=3.0)
  #show
  manager = plt.get_current_fig_manager()
  #manager.window.state('zoomed')
  JumpFrames = 0
  for j in range(stackFrames):
    # determine none-bkg pixels from uint16 matrix (all positive int. of
    # NP) under the assumption that most pixels are bkg
    averageUint=np.mean(np.mean(subtractedMatrix[j,:,:])) #average of the full frame subtracted matrix (uint16)
    stdevUint=np.std(np.std(subtractedMatrix[j,:,:].astype(np.float64)))
    # identify particles based on all pixels
    # Create logical mask by filtering by intensity and size
    print(averageUint + (threshStack * stdevUint))
    particleRegion[j, :, :] = subtractedMatrix[j, :, :] > 2*(averageUint + (threshStack * stdevUint))
    particleRegion[j,:,:] = remove_small_objects(particleRegion[j,:,:], min_size=particleSizeThresh)

    # extract possible particles within the ROI
    particleRegionROI[j, :, :] = particleRegion[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1]
    if j<50:
      plt.figure(figsize=(6, 6))
      plt.imshow(particleRegion[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('Particle Region')
      plt.show()

      plt.figure(figsize=(6, 6))
      plt.imshow(subtractedMatrix[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('Subtracted Matrix ROI')
      plt.show()
    # label the regions in the logical mask
    L, num_features = label(particleRegionROI[j, :, :])
    # get properties of the possible particles regions: the centroid and bounding box
    slices = find_objects(L)
    # calculate centroids and bounding boxes
    centroids = [center_of_mass(particleRegionROI[j, :, :], L, i + 1) for i in range(num_features)]
    bounding_boxes = [sli for sli in slices]
    # combine centroids and bounding boxes into particleProps
    particleProps = [{'Centroid': centroids[i], 'BoundingBox': bounding_boxes[i]} for i in range(num_features)]

    if j>1:
      previousCoord = particleCentroidROI[j-1, :]
    # if j<20:
    #   plt.figure(figsize=(6, 6))
    #   plt.imshow(subtractedMatrix[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1], cmap='gray')
    #   plt.colorbar(label='Value')  # Add a colorbar to indicate values
    #   plt.title('Subtracted Matrix ROI')
    #   plt.show()

    #   print(f"particleCentroidROI[j-1, :]: {particleCentroidROI[j-1, :]}")
    #   plt.figure(figsize=(6, 6))
    #   plt.imshow(particleRegionROI[j, :, :], cmap='gray')
    #   plt.colorbar(label='Value')  # Add a colorbar to indicate values
    #   plt.title('Boolean Matrix Plot')
    #   plt.show()

    # if more than one region identified, choose the one closest to
    # previous centroid (for confined and regular diffusion). For active
    # transport, this can be a problem
    print(f"len(particleProps): {len(particleProps)}")
    print(f'particle props: {particleProps}')
    if len(particleProps) >= 2:
      coordList = np.zeros((len(particleProps), 2))
      distances = np.zeros(len(particleProps))
      for h in range(len(particleProps)):
        # get the centroid coordinates
        coordList[h, :] = particleProps[h]['Centroid']
        # compute the distance between previousCoord and the current centroid
        distances[h] = pdist([previousCoord, coordList[h, :]], 'euclidean')[0]
        # update chosenParticle with the index of the minimum distance
        chosenParticle = np.argmin(distances[:h+1])
      particleProps = [particleProps[chosenParticle]]
      particleRegionROI[j, :, :] = (L==(chosenParticle+ 1))  # +1 because Python's label function starts from 1

    if len(particleProps) == 1:
      centroid = particleProps[0]['Centroid']
      print(f'centroid: {centroid}')
      particleCentroidROI[j, :] = centroid
      # if the particle jumped too far, set the centroid as last frame
      # location and define a 'default particle mask' centered around the
      # centroid.
      distanceFromPrev = pdist(np.array([previousCoord, particleCentroidROI[j, :]]))[0]
      if distanceFromPrev > jumpThresh:
        JumpFrames += 1
        particleCentroidROI[j, :] = previousCoord  # Assume particle didn't move from last frame
        # create a meshgrid for the ROI
        rr, cc = np.meshgrid(np.arange(1, 2*ROI+2), np.arange(1, 2*ROI+2))  # Adjust for Python's 0-indexing
        # update particleRegionROI to include the current mask
        particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2
        #check element wise power

    if len(particleProps) == 0:
      JumpFrames += 1
      particleCentroidROI[j, :] = previousCoord  # Assume particle didn't move from last frame
      rr, cc = np.meshgrid(np.arange(1, 2*ROI + 2), np.arange(1, 2*ROI + 2))  # Adjust for Python's 0-indexing
      particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2
    particlePropsFinal = regionprops(particleRegionROI[j, :, :].astype(int))

    if particlePropsFinal and particlePropsFinal[0].area < minArea:
      # create a meshgrid for the ROI
      rr, cc = np.meshgrid(np.arange(1, 2*ROI+2), np.arange(1, 2*ROI+2))  # Adjust for Python's 0-indexing

      # update particleRegionROI to include the current mask
      particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2
      if j<20:
        plt.figure(figsize=(6, 6))
        plt.imshow(particleRegionROI[j, :, :], cmap='gray')
        plt.colorbar(label='Value')
        plt.title('Particle Region ROI')
        plt.show()

  print(f'particle centroid roi: {particleCentroidROI}')
  centroidGlobalColX = np.round(particleCentroidROI[:, 0]).astype(int) + (startColX - (ROI + 1))
  centroidGlobalRowY = np.round(particleCentroidROI[:, 1]).astype(int) + (startRowY - (ROI + 1))

  T = pd.DataFrame({
      'centroidGlobalColX': centroidGlobalColX,
      'centroidGlobalRowY': centroidGlobalRowY
  })

  # print the DataFrame to verify the contents
  print(T)

  # save the DataFrame to an Excel file
  T.to_excel(f'{saveBaseName}_Coords.xlsx', index=False)

  #calculation
  rawROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=np.int16)
  nonebkgROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=bool)
  bkgregionROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=bool)
  localAverage = np.zeros(stackFrames)
  localStdev = np.zeros(stackFrames)
  particleSignal = np.zeros(stackFrames)
  particleSignalBright = np.zeros(stackFrames)
  particleSignalDark = np.zeros(stackFrames)
  nanCountBrights = np.zeros(stackFrames)
  nanCountDarks = np.zeros(stackFrames)
  numberBrightPixels = np.zeros(stackFrames)
  numberDarkPixels = np.zeros(stackFrames)

  for j in range(stackFrames):
    rawROI[j, :, :] = stack[j, startRowY - ROI:startRowY + ROI + 1, startColX - ROI:startColX + ROI + 1]
    rawROIHolder = rawROI[j, :, :]
    averageROI = np.mean(rawROIHolder)
    NPidentifyROI = filters.median(rawROIHolder - averageROI) + filters.median(averageROI - rawROIHolder)
    if j<20:
      plt.figure(figsize=(6, 6))
      plt.imshow(rawROIHolder, cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('Particle Region ROI')
      plt.show()

      plt.figure(figsize=(6, 6))
      plt.imshow(NPidentifyROI, cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('NPidentifyROI')
      plt.show()

    averageFrame = np.mean(NPidentifyROI)
    stdevFrame = np.std(NPidentifyROI)
    nonebkgROI[j, :, :] = morphology.remove_small_objects(NPidentifyROI > (averageFrame + threshFrame * stdevFrame), particleSizeThresh)
    bkgregionROI[j, :, :] = ~particleRegion[j, startRowY - ROI:startRowY + ROI + 1, startColX - ROI:startColX + ROI + 1]
    if j<20:

      plt.figure(figsize=(6, 6))
      plt.imshow(particleRegion[j, :, :], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('particleregionROI')
      plt.show()

      plt.figure(figsize=(6, 6))
      plt.imshow(bkgregionROI[j, :, :], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('bkg region ROI')
      plt.show()

    particleCentroidROIround = np.round(particleCentroidROI[j, :])#.astype(int)

    if (particleCentroidROIround[0] - ROISmall < 0 or particleCentroidROIround[1] - ROISmall < 0 or
            particleCentroidROIround[0] + ROISmall >= 2 * ROI + 1 or particleCentroidROIround[1] + ROISmall >= 2 * ROI + 1):
        print('Index Error')
        continue

    # rawROIholderLocal = rawROI[
    #     j,
    #     particleCentroidROIround[1] - ROISmall:particleCentroidROIround[1] + ROISmall + 1,
    #     particleCentroidROIround[0] - ROISmall:particleCentroidROIround[0] + ROISmall + 1
    # ]
    # notParticlesLocal = bkgregionROI[
    #     j,
    #     particleCentroidROIround[1] - ROISmall:particleCentroidROIround[1] + ROISmall + 1,
    #     particleCentroidROIround[0] - ROISmall:particleCentroidROIround[0] + ROISmall + 1
    # ]

    if np.any(bkgregionROI):  # check if there are any true elements
      localAverage[j] = np.mean(rawROI[bkgregionROI])
      localStdev[j] = np.std(rawROI[bkgregionROI])
    else:
      localAverage[j] = 0
      localStdev[j] = 0

    particleSignal[j] = np.mean(rawROIHolder[particleRegionROI[j, :, :]])

    particleSignalList = rawROIHolder[particleRegionROI[j, :, :]]
    particleSignalListBright = particleSignalList > localAverage[j]
    particleSignalListDark = particleSignalList < localAverage[j]
    numberBrightPixels[j] = np.sum(particleSignalListBright)
    numberDarkPixels[j] = np.sum(particleSignalListDark)

    if np.any(particleSignalListBright):  # check if there are any true elements
        particleSignalBright[j] = np.mean(particleSignalList[particleSignalListBright])
    else:
        particleSignalBright[j] = localAverage[j]
        nanCountBrights[j] = 1

    if np.any(particleSignalListDark):  # check if there are any true elements
        particleSignalDark[j] = np.mean(particleSignalList[particleSignalListDark])
    else:
        particleSignalDark[j] = localAverage[j]
        nanCountDarks[j] = 1

  brightContrast = (particleSignalBright - localAverage) / localAverage
  darkContrast = (particleSignalDark - localAverage) / localAverage
  brightContrastNorm = (brightContrast - np.min(brightContrast)) / (np.max(brightContrast) - np.min(brightContrast))
  darkContrastNorm = (darkContrast - np.max(darkContrast)) / (np.min(darkContrast) - np.max(darkContrast))
  contrastDifference = brightContrastNorm - darkContrastNorm
  contrast = (particleSignal - localAverage) / localAverage

  particleCentroidROIColXSmooth = gaussian_filter(particleCentroidROI[:, 1], sigma=2)
  particleCentroidROIRowYSmooth = gaussian_filter(particleCentroidROI[:, 0], sigma=2)
  centroidGlobalColXSmooth = gaussian_filter(centroidGlobalColX, sigma=2)
  centroidGlobalRowYSmooth = gaussian_filter(centroidGlobalRowY, sigma=2)

  positionColX_um = centroidGlobalColX * pixelSize
  positionRowY_um = centroidGlobalRowY * pixelSize
  positionColXFirst = positionColX_um[0]
  positionRowYFirst = positionRowY_um[0]
  displacement_um = np.sqrt((positionColX_um - positionColXFirst) ** 2 + (positionRowY_um - positionRowYFirst) ** 2)
  displacement_umSmooth = gaussian_filter(displacement_um, sigma=2)

  #saving results
  frameNum = np.arange(1, stackFrames + 1)
  timeStampExp = np.arange(0, stackFrames * exposureTime, exposureTime)

  result_df = pd.DataFrame({
    'frameNum': frameNum, 'timeStampExp': timeStampExp, 'localAverage': localAverage,
    'localStdev': localStdev, 'particleSignal': particleSignal, 'contrast': contrast,
    'particleSignalBright': particleSignalBright, 'particleSignalDark': particleSignalDark,
    'brightContrast': brightContrast, 'darkContrast': darkContrast, 'brightContrastNorm': brightContrastNorm,
    'darkContrastNorm': darkContrastNorm, 'contrastDifference': contrastDifference,
    'particleCentroidROIX': particleCentroidROI[:, 1], 'particleCentroidROIY': particleCentroidROI[:, 0],
    'particleCentroidROIColXSmooth': particleCentroidROIColXSmooth, 'particleCentroidROIRowYSmooth': particleCentroidROIRowYSmooth,
    'centroidGlobalColX': centroidGlobalColX, 'centroidGlobalRowY': centroidGlobalRowY,
    'centroidGlobalColXSmooth': centroidGlobalColXSmooth, 'centroidGlobalRowYSmooth': centroidGlobalRowYSmooth,
    'displacement_um': displacement_um, 'displacement_umSmooth': displacement_umSmooth
  })
  result_df.to_excel(f'{saveBaseName}_Result.xlsx', index=False)

  parameters_df = pd.DataFrame({
    'imageFolder': [stackPath], 'imageFile': [stackFile], 'SFrame': [SFrame], 'EFrame': [EFrame],
    'startColX': [startColX], 'startRowY': [startRowY], 'analyzeMode': [analyzeMode], 'ROI': [ROI],
    'ROISmall': [ROISmall], 'frameRange': [frameRange], 'threshStack': [threshStack], 'threshFrame': [threshFrame],
    'particleSizeThresh': [particleSizeThresh], 'particleDefaultDiameter': [particleDefaultDiameter],
    'minArea': [minArea], 'jumpThresh': [jumpThresh], 'exposureTime': [exposureTime], 'pixelSize': [pixelSize]
  })
  parameters_df.to_excel(f'{saveBaseName}_Parameters.xlsx', index=False)

  rawROI_uint16 = rawROI.astype(np.int16)
  io.imsave(f'{saveBaseName}_ROI.tif', rawROI)

  particleidentified = np.zeros_like(rawROI)
  vmin = 6500
  vmax = 7500
  image_paths = []
  for h in range(stackFrames):
    if h < 10:
      plt.imshow(rawROI[h, :, :], cmap='gray')
      plt.colorbar()
      plt.title('TIFF Image with Colorbar')
      plt.show()
    roiframe = rawROI[h, :, :] # unsure why this is necessary
    particleframe = np.zeros_like(roiframe)
    particleframe[particleRegionROI[h, :, :]] = roiframe[particleRegionROI[h, :, :]]
    particleframe[~particleRegionROI[h, :, :]] = localAverage[h] # - 100
    if h < 10:
      plt.imshow(particleframe, cmap='gray', vmin=vmin, vmax=vmax)
      plt.colorbar()
      plt.title('TIFF Image with Colorbar')
      plt.show()

    image_path = f'particle_frame_{h+1}.png'
    plt.savefig(image_path)
    image_paths.append(image_path)
    plt.close()

    particleframe = np.clip(particleframe, vmin, vmax)  # Ensure values are within the desired range
    particleidentified[h, :, :] = particleframe

  images = [Image.open(image_path) for image_path in image_paths]
  images[0].save(f'{saveBaseNameMovie}_Particle_{particleID}.tif', save_all=True, append_images=images[1:])


  # io.imsave(f'{saveBaseNameMovie}_Particle {particleID}.tif', particleidentified)

  #plots
  plt.figure()
  plt.plot(timeStampExp, localAverage, label='Local background')
  plt.plot(timeStampExp, localAverage + localStdev, label='+ Local stdev')
  plt.plot(timeStampExp, localAverage - localStdev, label='- Local stdev')
  plt.plot(timeStampExp, particleSignal, label='Particle signal')
  plt.title(f'Mean Signal, file {saveID}, Particle {particleID}, StartX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Intensity')
  plt.savefig(f'{saveBaseName}_SignalMean.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, localAverage, label='Local background')
  plt.plot(timeStampExp, localAverage + localStdev, label='+ Local stdev')
  plt.plot(timeStampExp, localAverage - localStdev, label='- Local stdev')
  plt.plot(timeStampExp, particleSignalBright, label='Bright signal')
  plt.plot(timeStampExp, particleSignalDark, label='Dark signal')
  plt.title(f'Bright and Dark Signal, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Intensity')
  plt.savefig(f'{saveBaseName}_SignalBrightDark.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, brightContrast, label='Bright Contrast')
  plt.plot(timeStampExp, darkContrast, label='Dark Contrast')
  plt.title(f'Bright and Dark Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast')
  plt.savefig(f'{saveBaseName}_BD_Contrast.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, brightContrastNorm, label='Normalized Bright Contrast')
  plt.plot(timeStampExp, darkContrastNorm, label='Normalized Dark Contrast')
  plt.title(f'Normalized Bright and Dark Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Normalized Contrast')
  plt.savefig(f'{saveBaseName}_ContrastNorm.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, contrastDifference)
  plt.plot(timeStampExp, np.zeros(stackFrames), label='Zero line')
  plt.title(f'Contrast Difference, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast Difference')
  plt.ylim([-1, 1])
  plt.savefig(f'{saveBaseName}_ContrastDiff.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, contrast)
  plt.plot(timeStampExp, np.zeros(stackFrames), label='Zero line')
  plt.title(f'Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast')
  plt.savefig(f'{saveBaseName}_Contrast.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, centroidGlobalColX, label='X position')
  plt.plot(timeStampExp, centroidGlobalColXSmooth, label='X position smooth')
  plt.title(f'X position, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('X position (pixels)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_positionX.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, centroidGlobalRowY, label='Y position')
  plt.plot(timeStampExp, centroidGlobalRowYSmooth, label='Y position smooth')
  plt.title(f'Y position, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Y position (pixels)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_positionY.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, displacement_um, label='Displacement')
  plt.plot(timeStampExp, displacement_umSmooth, label='Displacement smooth')
  plt.title(f'Displacement (um), file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Displacement (um)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_Displacement.eps')
  plt.show()

  #overlay figures
  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.scatter(centroidGlobalColX[0], centroidGlobalRowY[0], color='red')
  plt.plot(centroidGlobalColX, centroidGlobalRowY, color='yellow')
  plt.title(f'Overlay, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_Overlay.eps')
  plt.show()

  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.scatter(centroidGlobalColX[0], centroidGlobalRowY[0], color='red')
  plt.plot(centroidGlobalColXSmooth, centroidGlobalRowYSmooth, color='yellow')
  plt.title(f'Overlay Smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlaySmooth.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.plot(particleCentroidROI[:, 1], particleCentroidROI[:, 0], color='yellow')
  plt.title(f'Overlay zoom, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlayZoom.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.plot(particleCentroidROIColXSmooth, particleCentroidROIRowYSmooth, color='yellow')
  plt.title(f'Overlay zoom smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlayZoomSmooth.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(centroidGlobalColX))
  plt.scatter(centroidGlobalColX, centroidGlobalRowY, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(centroidGlobalRowY[0], centroidGlobalColX[0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'Full path, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathFull.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(particleCentroidROI))
  plt.scatter(particleCentroidROI[:, 1], particleCentroidROI[:, 0], c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'ROI path, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathROI.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(centroidGlobalColXSmooth))
  plt.scatter(centroidGlobalColXSmooth, centroidGlobalRowYSmooth, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(centroidGlobalRowY[0], centroidGlobalColX[0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'Full path smooth, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathFullSmooth.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(particleCentroidROIColXSmooth))
  plt.scatter(particleCentroidROIColXSmooth, particleCentroidROIRowYSmooth, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'ROI path smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathROISmooth.eps')
  plt.show()

  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.title(f'Frame {SFrame}, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.savefig(f'{saveBaseName}_Frame{SFrame}.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.title(f'Raw ROI, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.savefig(f'{saveBaseName}_Frame1RawROI.eps')
  plt.show()

  plt.close('all')

DICAnalysis('/content/drive/MyDrive/Colab Notebooks/Copy of Cell1_S1.tif', 'P1_C1_S1_11', 440, 281, 1, 300)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Code for weighted bright and dark matrices**

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, morphology, filters, measure
from skimage.morphology import remove_small_objects
from skimage.measure import regionprops
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.ndimage import label
from scipy.ndimage import center_of_mass, find_objects
from scipy.spatial.distance import pdist
import cv2
from PIL import Image
import math
from scipy.signal import wiener
import tifffile as tiff


def DICAnalysis(Path, ID, X, Y, Sframe, Eframe):
  stackPath, stackFile = os.path.split(Path)

  startColX = X
  startRowY = Y
  particleID = ID
  SFrame = Sframe
  EFrame = Eframe

  analyzeMode = "track"

  # processing paremetersok
  frameRange = 20 #used in moving average
  threshStack = 10 #threshold for indentifying none-bkg pixels
  threshFrame = 0.6 #threshole for indentifying none-bkw pixeld within ROI
  particleSizeThresh = 20 #number of pixels in particle
  jumpThresh = 20 #longest distance allowen in adjacent thresh for NP tracking
  ROI = 50 #ROI around particle
  ROISmall = 18 # region around particle
  particleDefaultDiameter = 10
  minArea = np.floor(np.pi*(particleDefaultDiameter/2)**2) #pixels

  #camera parameters
  exposureTime = 0.100
  pixelSize = 0.065

  #set up for saving
  saveID = os.path.splitext(stackFile)[0]
  savingPath = f'{stackPath}/Process/Particle {particleID}/'
  if not os.path.isdir(savingPath):
    os.makedirs(savingPath)
  saveBaseName = f'{savingPath}{saveID}_Particle{particleID}_frameRange{frameRange}_X{startColX}_Y{startRowY}'
  saveBaseNameMovie = f'{savingPath}{saveID}_frameRange{frameRange}'
  print(f"saveID: {saveID}")
  print(f"savingPath: {savingPath}")
  print(f"saveBaseName: {saveBaseName}")
  print(f"saveBaseNameMovie: {saveBaseNameMovie}")

  # read image
  stackName = os.path.join(stackPath, stackFile)

  stackFrames = EFrame-SFrame+1

  stack = io.imread(stackName)

  # Get the number of frames and image dimensions
  stackFrames = stack.shape[0]
  height, width = stack.shape[1], stack.shape[2]

  print(f"Initial stackFrames: {stackFrames}")
  print(f"height: {height}, width: {width}")

  # Adjust the number of frames if EFrame is not finite
  if not math.isfinite(EFrame):
    EFrame = stackFrames

  #stackFrames = EFrame - SFrame + 1
  print(f"stackFrames: {stackFrames}")

  # Initialize stackMatrix
  stackMatrix = np.zeros((stackFrames, height, width), dtype=np.uint16)
  for i in range(stackFrames):
    stackMatrix[i, :, :] = stack[i + SFrame - 1]

  #pre allocate matrices
  subtractedMatrixDouble = np.zeros_like(stackMatrix, dtype=np.float64)  # values will be doubles
  subtractedMatrix = np.zeros_like(stackMatrix, dtype=np.int16)  # values will be unsigned integers
  subtractedMatrixB = np.zeros_like(stackMatrix, dtype=np.int16)
  subtractedMatrixD = np.zeros_like(stackMatrix, dtype=np.int16)
  for i in range(stackFrames):
    #calculate moving average
    startFrame = max(i - frameRange // 2, 1)
    endFrame = min(startFrame + frameRange, stackFrames)
    if (endFrame - startFrame) < (frameRange - 1):
      startFrame = endFrame - frameRange
    averageImage = np.mean(stackMatrix[startFrame:endFrame, :, :], axis=0).astype(np.uint16) #mean along 3rd dimention (depth/time)
    #Calculate subtracted image as a double so negative values do not get deleted; aka preserves bright/dark contrast over middle background
    imageMatrixDouble = stackMatrix[i, :, :].astype(np.float64)
    subtractedMatrixDouble[i, :, :] = imageMatrixDouble - averageImage.astype(np.float64)
    subtractedMatrix[i, :, :] = np.abs(subtractedMatrixDouble[i, :, :]).astype(np.uint16)

    subtractedMatrixB[i, :, :] = (stackMatrix[i, :, :] - (averageImage))
    subtractedMatrixD[i, :, :] = (averageImage - stackMatrix[i, :, :])
    subtractedMatrixB = np.where(subtractedMatrixB > 0, subtractedMatrixB * 2, subtractedMatrixB)
    subtractedMatrixD = np.where(subtractedMatrixD > 0, subtractedMatrixD * 2, subtractedMatrixD)
    subtractedMatrix[i, :, :] = np.add(subtractedMatrixB[i, :, :], subtractedMatrixD[i, :, :])  # this movie will be bright where there are particles and dark everywhere else, for localization purposes, not intensity.

  minVal= np.abs(np.min(np.min(np.min(subtractedMatrixDouble)))) #get the darkest pixel in all frames
  subtractedMatrixDoubleSaving = (subtractedMatrixDouble + minVal + 100).astype(np.uint16) #make the lowest pixel intensity as 100

  subMatrixZerosTest = np.zeros_like(subtractedMatrix)
  minVal= np.abs(np.min(np.min(np.min(subtractedMatrixDouble)))) #get the darkest pixel in all frames
  subtractedMatrixDoubleSaving = (subtractedMatrixDouble + minVal + 100).astype(np.uint16) #make the lowest pixel intensity as 100

  # auto tracking
  # preallocate matrices
  particleRegionROI = np.zeros((stackFrames, 2*ROI + 1, 2*ROI+ 1), dtype=bool)
  particleRegion = np.zeros_like(subtractedMatrix, dtype=bool)  # label the possible particles (including vesicles) within the whole image
  particleCentroidROI = np.zeros((stackFrames, 2))
  particleRegionD = np.zeros_like(subtractedMatrix, dtype=bool)  # label the possible particles (including vesicles) within the whole image
  particleRegionB = np.zeros_like(subtractedMatrix, dtype=bool)  # label the possible particles (including vesicles) within the whole image
  previousCoord = [ROI+1, ROI+1]
  fig = plt.figure()
  fig.set_size_inches(10.5, 10.5, forward=True)
  fig.tight_layout(pad=3.0)
  #show
  manager = plt.get_current_fig_manager()
  #manager.window.state('zoomed')
  JumpFrames = 0
  for j in range(stackFrames):
    # determine none-bkg pixels from uint16 matrix (all positive int. of
    # NP) under the assumption that most pixels are bkg
    averageUint=np.mean(np.mean(subtractedMatrix[j,:,:])) #average of the full frame subtracted matrix (uint16)
    stdevUint=np.std(np.std(subtractedMatrix[j,:,:].astype(np.float64)))
    # identify particles based on all pixels
    # Create logical mask by filtering by intensity and size
    particleRegionD[j,:,:] = subtractedMatrixD[j, :, :] > (averageUint + (threshStack * stdevUint))
    particleRegionD[j,:,:] = remove_small_objects(particleRegionD[j,:,:], min_size=particleSizeThresh)

    particleRegionB[j,:,:] = subtractedMatrixB[j, :, :] > (averageUint + (threshStack * stdevUint))
    particleRegionB[j,:,:] = remove_small_objects(particleRegionB[j,:,:], min_size=particleSizeThresh)

    particleRegion[j, :, :] = particleRegionD[j,:,:]+particleRegionB[j,:,:]
    # extract possible particles within the ROI
    particleRegionROI[j, :, :] = particleRegion[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1]
    # label the regions in the logical mask
    L, num_features = label(particleRegionROI[j, :, :])
    # get properties of the possible particles regions: the centroid and bounding box
    slices = find_objects(L)
    # calculate centroids and bounding boxes
    centroids = [center_of_mass(particleRegionROI[j, :, :], L, i + 1) for i in range(num_features)]
    bounding_boxes = [sli for sli in slices]
    # combine centroids and bounding boxes into particleProps
    particleProps = [{'Centroid': centroids[i], 'BoundingBox': bounding_boxes[i]} for i in range(num_features)]

    if j>1:
      previousCoord = particleCentroidROI[j-1, :]
    if j<10:
      plt.figure(figsize=(6, 6))
      plt.imshow(subtractedMatrix[j, startRowY-ROI:startRowY+ROI+1, startColX-ROI:startColX+ROI+1], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('Subtracted Matrix ROI')
      plt.show()

      print(f"particleCentroidROI[j-1, :]: {particleCentroidROI[j-1, :]}")
      plt.figure(figsize=(6, 6))
      plt.imshow(particleRegionROI[j, :, :], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('Boolean Matrix Plot')
      plt.show()

    # if more than one region identified, choose the one closest to
    # previous centroid (for confined and regular diffusion). For active
    # transport, this can be a problem
    #print(f"len(particleProps): {len(particleProps)}")
    print(f'particle props: {particleProps}')
    if len(particleProps) >= 2:
      coordList = np.zeros((len(particleProps), 2))
      distances = np.zeros(len(particleProps))
      for h in range(len(particleProps)):
        # get the centroid coordinates
        coordList[h, :] = particleProps[h]['Centroid']
        # compute the distance between previousCoord and the current centroid
        distances[h] = pdist([previousCoord, coordList[h, :]], 'euclidean')[0]
        # update chosenParticle with the index of the minimum distance
        chosenParticle = np.argmin(distances[:h+1])
      particleProps = [particleProps[chosenParticle]]
      particleRegionROI[j, :, :] = (L == (chosenParticle + 1))  # +1 because Python's label function starts from 1

    if len(particleProps) == 1:
      centroid = particleProps[0]['Centroid']
      print(f'centroid: {centroid}')
      particleCentroidROI[j, :] = centroid
      # if the particle jumped too far, set the centroid as last frame
      # location and define a 'default particle mask' centered around the
      # centroid.
      distanceFromPrev = pdist(np.array([previousCoord, particleCentroidROI[j, :]]))[0]
      if distanceFromPrev > jumpThresh:
        JumpFrames += 1
        particleCentroidROI[j, :] = previousCoord  # Assume particle didn't move from last frame
        # create a meshgrid for the ROI
        rr, cc = np.meshgrid(np.arange(1, 2*ROI+2), np.arange(1, 2*ROI+2))  # Adjust for Python's 0-indexing
        # update particleRegionROI to include the current mask
        particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2
        #check element wise power

    if len(particleProps) == 0:
      JumpFrames += 1
      particleCentroidROI[j, :] = previousCoord  # Assume particle didn't move from last frame
      rr, cc = np.meshgrid(np.arange(1, 2*ROI + 2), np.arange(1, 2*ROI + 2))  # Adjust for Python's 0-indexing
      particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2
    particlePropsFinal = regionprops(particleRegionROI[j, :, :].astype(int))

    if particlePropsFinal and particlePropsFinal[0].area < minArea:
      # create a meshgrid for the ROI
      rr, cc = np.meshgrid(np.arange(1, 2*ROI+2), np.arange(1, 2*ROI+2))  # Adjust for Python's 0-indexing

      # update particleRegionROI to include the current mask
      particleRegionROI[j, :, :] = ((rr - particleCentroidROI[j, 0])**2 + (cc - particleCentroidROI[j, 1])**2) < (particleDefaultDiameter / 2)**2

  print(f'particle centroid roi: {particleCentroidROI}')
  centroidGlobalColX = np.round(particleCentroidROI[:, 0]).astype(int) + (startColX - (ROI + 1))
  centroidGlobalRowY = np.round(particleCentroidROI[:, 1]).astype(int) + (startRowY - (ROI + 1))

  T = pd.DataFrame({
      'centroidGlobalColX': centroidGlobalColX,
      'centroidGlobalRowY': centroidGlobalRowY
  })

  # print the DataFrame to verify the contents
  print(T)

  # save the DataFrame to an Excel file
  T.to_excel(f'{saveBaseName}_Coords.xlsx', index=False)

  #calculation
  rawROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=np.int16)
  nonebkgROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=bool)
  bkgregionROI = np.zeros((stackFrames, 2 * ROI + 1, 2 * ROI + 1), dtype=bool)
  localAverage = np.zeros(stackFrames)
  localStdev = np.zeros(stackFrames)
  particleSignal = np.zeros(stackFrames)
  particleSignalBright = np.zeros(stackFrames)
  particleSignalDark = np.zeros(stackFrames)
  nanCountBrights = np.zeros(stackFrames)
  nanCountDarks = np.zeros(stackFrames)
  numberBrightPixels = np.zeros(stackFrames)
  numberDarkPixels = np.zeros(stackFrames)

  for j in range(stackFrames):
    rawROI[j, :, :] = stack[j, startRowY - ROI:startRowY + ROI + 1, startColX - ROI:startColX + ROI + 1]
    rawROIHolder = rawROI[j, :, :]
    averageROI = np.mean(rawROIHolder)
    NPidentifyROI = filters.median(rawROIHolder - averageROI) + filters.median(averageROI - rawROIHolder)
    if j < 20:
      plt.figure(figsize=(6, 6))
      plt.imshow(NPidentifyROI, cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('NPidentifyROI')
      plt.show()
    averageFrame = np.mean(NPidentifyROI)
    stdevFrame = np.std(NPidentifyROI)
    nonebkgROI[j, :, :] = morphology.remove_small_objects(NPidentifyROI > (averageFrame + threshFrame * stdevFrame), particleSizeThresh)
    bkgregionROI[j, :, :] = ~nonebkgROI[j, :, :] & ~particleRegionROI[j, :, :]
    if j < 20:
      plt.figure(figsize=(6, 6))
      plt.imshow(nonebkgROI[j, :, :], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('non bkg ROI')
      plt.show()

      plt.figure(figsize=(6, 6))
      plt.imshow(bkgregionROI[j, :, :], cmap='gray')
      plt.colorbar(label='Value')  # Add a colorbar to indicate values
      plt.title('bkg ROI')
      plt.show()

    particleCentroidROIround = np.round(particleCentroidROI[j, :]).astype(int)

    if (particleCentroidROIround[0] - ROISmall < 0 or particleCentroidROIround[1] - ROISmall < 0 or
            particleCentroidROIround[0] + ROISmall >= 2 * ROI + 1 or particleCentroidROIround[1] + ROISmall >= 2 * ROI + 1):
        print('Index Error')
        continue

    rawROIholderLocal = rawROI[
        j,
        particleCentroidROIround[1] - ROISmall:particleCentroidROIround[1] + ROISmall + 1,
        particleCentroidROIround[0] - ROISmall:particleCentroidROIround[0] + ROISmall + 1
    ]
    notParticlesLocal = bkgregionROI[
        j,
        particleCentroidROIround[1] - ROISmall:particleCentroidROIround[1] + ROISmall + 1,
        particleCentroidROIround[0] - ROISmall:particleCentroidROIround[0] + ROISmall + 1
    ]

    if np.any(notParticlesLocal):  # check if there are any true elements
      localAverage[j] = np.mean(rawROIholderLocal[notParticlesLocal])
      localStdev[j] = np.std(rawROIholderLocal[notParticlesLocal])
    else:
      localAverage[j] = 0
      localStdev[j] = 0

    particleSignal[j] = np.mean(rawROIHolder[particleRegionROI[j, ...]])

    particleSignalList = rawROIHolder[particleRegionROI[j, ...]]
    particleSignalListBright = particleSignalList > localAverage[j]
    particleSignalListDark = particleSignalList < localAverage[j]
    numberBrightPixels[j] = np.sum(particleSignalListBright)
    numberDarkPixels[j] = np.sum(particleSignalListDark)

    if np.any(particleSignalListBright):  # check if there are any true elements
        particleSignalBright[j] = np.mean(particleSignalList[particleSignalListBright])
    else:
        particleSignalBright[j] = localAverage[j]
        nanCountBrights[j] = 1

    if np.any(particleSignalListDark):  # check if there are any true elements
        particleSignalDark[j] = np.mean(particleSignalList[particleSignalListDark])
    else:
        particleSignalDark[j] = localAverage[j]
        nanCountDarks[j] = 1

  brightContrast = (particleSignalBright - localAverage) / localAverage
  darkContrast = (particleSignalDark - localAverage) / localAverage
  brightContrastNorm = (brightContrast - np.min(brightContrast)) / (np.max(brightContrast) - np.min(brightContrast))
  darkContrastNorm = (darkContrast - np.max(darkContrast)) / (np.min(darkContrast) - np.max(darkContrast))
  contrastDifference = brightContrastNorm - darkContrastNorm
  contrast = (particleSignal - localAverage) / localAverage

  particleCentroidROIColXSmooth = gaussian_filter(particleCentroidROI[:, 1], sigma=2)
  particleCentroidROIRowYSmooth = gaussian_filter(particleCentroidROI[:, 0], sigma=2)
  centroidGlobalColXSmooth = gaussian_filter(centroidGlobalColX, sigma=2)
  centroidGlobalRowYSmooth = gaussian_filter(centroidGlobalRowY, sigma=2)

  positionColX_um = centroidGlobalColX * pixelSize
  positionRowY_um = centroidGlobalRowY * pixelSize
  positionColXFirst = positionColX_um[0]
  positionRowYFirst = positionRowY_um[0]
  displacement_um = np.sqrt((positionColX_um - positionColXFirst) ** 2 + (positionRowY_um - positionRowYFirst) ** 2)
  displacement_umSmooth = gaussian_filter(displacement_um, sigma=2)

  #saving results
  frameNum = np.arange(1, stackFrames + 1)
  timeStampExp = np.arange(0, stackFrames * exposureTime, exposureTime)

  result_df = pd.DataFrame({
    'frameNum': frameNum, 'timeStampExp': timeStampExp, 'localAverage': localAverage,
    'localStdev': localStdev, 'particleSignal': particleSignal, 'contrast': contrast,
    'particleSignalBright': particleSignalBright, 'particleSignalDark': particleSignalDark,
    'brightContrast': brightContrast, 'darkContrast': darkContrast, 'brightContrastNorm': brightContrastNorm,
    'darkContrastNorm': darkContrastNorm, 'contrastDifference': contrastDifference,
    'particleCentroidROIX': particleCentroidROI[:, 1], 'particleCentroidROIY': particleCentroidROI[:, 0],
    'particleCentroidROIColXSmooth': particleCentroidROIColXSmooth, 'particleCentroidROIRowYSmooth': particleCentroidROIRowYSmooth,
    'centroidGlobalColX': centroidGlobalColX, 'centroidGlobalRowY': centroidGlobalRowY,
    'centroidGlobalColXSmooth': centroidGlobalColXSmooth, 'centroidGlobalRowYSmooth': centroidGlobalRowYSmooth,
    'displacement_um': displacement_um, 'displacement_umSmooth': displacement_umSmooth
  })
  result_df.to_excel(f'{saveBaseName}_Result.xlsx', index=False)

  parameters_df = pd.DataFrame({
    'imageFolder': [stackPath], 'imageFile': [stackFile], 'SFrame': [SFrame], 'EFrame': [EFrame],
    'startColX': [startColX], 'startRowY': [startRowY], 'analyzeMode': [analyzeMode], 'ROI': [ROI],
    'ROISmall': [ROISmall], 'frameRange': [frameRange], 'threshStack': [threshStack], 'threshFrame': [threshFrame],
    'particleSizeThresh': [particleSizeThresh], 'particleDefaultDiameter': [particleDefaultDiameter],
    'minArea': [minArea], 'jumpThresh': [jumpThresh], 'exposureTime': [exposureTime], 'pixelSize': [pixelSize]
  })
  parameters_df.to_excel(f'{saveBaseName}_Parameters.xlsx', index=False)

  rawROI_uint16 = rawROI.astype(np.uint16)
  io.imsave(f'{saveBaseName}_ROI.tif', rawROI_uint16)

  particleidentified = np.zeros_like(rawROI_uint16)
  for h in range(stackFrames):
    roiframe = rawROI_uint16[h, ...]
    particleframe = np.zeros_like(roiframe)
    particleframe[particleRegionROI[h, ...]] = roiframe[particleRegionROI[h, ...]]
    particleframe[~particleRegionROI[h, ...]] = localAverage[h] - 100
    particleidentified[h, ...] = particleframe

  io.imsave(f'{saveBaseNameMovie}_Particle {particleID}.tif', particleidentified)

  #plots
  plt.figure()
  plt.plot(timeStampExp, localAverage, label='Local background')
  plt.plot(timeStampExp, localAverage + localStdev, label='+ Local stdev')
  plt.plot(timeStampExp, localAverage - localStdev, label='- Local stdev')
  plt.plot(timeStampExp, particleSignal, label='Particle signal')
  plt.title(f'Mean Signal, file {saveID}, Particle {particleID}, StartX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Intensity')
  plt.savefig(f'{saveBaseName}_SignalMean.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, localAverage, label='Local background')
  plt.plot(timeStampExp, localAverage + localStdev, label='+ Local stdev')
  plt.plot(timeStampExp, localAverage - localStdev, label='- Local stdev')
  plt.plot(timeStampExp, particleSignalBright, label='Bright signal')
  plt.plot(timeStampExp, particleSignalDark, label='Dark signal')
  plt.title(f'Bright and Dark Signal, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Intensity')
  plt.savefig(f'{saveBaseName}_SignalBrightDark.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, brightContrast, label='Bright Contrast')
  plt.plot(timeStampExp, darkContrast, label='Dark Contrast')
  plt.title(f'Bright and Dark Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast')
  plt.savefig(f'{saveBaseName}_BD_Contrast.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, brightContrastNorm, label='Normalized Bright Contrast')
  plt.plot(timeStampExp, darkContrastNorm, label='Normalized Dark Contrast')
  plt.title(f'Normalized Bright and Dark Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.legend()
  plt.xlabel('Time (sec)')
  plt.ylabel('Normalized Contrast')
  plt.savefig(f'{saveBaseName}_ContrastNorm.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, contrastDifference)
  plt.plot(timeStampExp, np.zeros(stackFrames), label='Zero line')
  plt.title(f'Contrast Difference, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast Difference')
  plt.ylim([-1, 1])
  plt.savefig(f'{saveBaseName}_ContrastDiff.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, contrast)
  plt.plot(timeStampExp, np.zeros(stackFrames), label='Zero line')
  plt.title(f'Contrast, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Contrast')
  plt.savefig(f'{saveBaseName}_Contrast.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, centroidGlobalColX, label='X position')
  plt.plot(timeStampExp, centroidGlobalColXSmooth, label='X position smooth')
  plt.title(f'X position, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('X position (pixels)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_positionX.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, centroidGlobalRowY, label='Y position')
  plt.plot(timeStampExp, centroidGlobalRowYSmooth, label='Y position smooth')
  plt.title(f'Y position, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Y position (pixels)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_positionY.eps')
  plt.show()

  plt.figure()
  plt.plot(timeStampExp, displacement_um, label='Displacement')
  plt.plot(timeStampExp, displacement_umSmooth, label='Displacement smooth')
  plt.title(f'Displacement (um), file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('Time (sec)')
  plt.ylabel('Displacement (um)')
  plt.legend()
  plt.savefig(f'{saveBaseName}_Displacement.eps')
  plt.show()

  #overlay figures
  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.scatter(centroidGlobalColX[0], centroidGlobalRowY[0], color='red')
  plt.plot(centroidGlobalColX, centroidGlobalRowY, color='yellow')
  plt.title(f'Overlay, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_Overlay.eps')
  plt.show()

  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.scatter(centroidGlobalColX[0], centroidGlobalRowY[0], color='red')
  plt.plot(centroidGlobalColXSmooth, centroidGlobalRowYSmooth, color='yellow')
  plt.title(f'Overlay Smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlaySmooth.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.plot(particleCentroidROI[:, 1], particleCentroidROI[:, 0], color='yellow')
  plt.title(f'Overlay zoom, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlayZoom.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.plot(particleCentroidROIColXSmooth, particleCentroidROIRowYSmooth, color='yellow')
  plt.title(f'Overlay zoom smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_OverlayZoomSmooth.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(centroidGlobalColX))
  plt.scatter(centroidGlobalColX, centroidGlobalRowY, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(centroidGlobalRowY[0], centroidGlobalColX[0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'Full path, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathFull.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(particleCentroidROI))
  plt.scatter(particleCentroidROI[:, 1], particleCentroidROI[:, 0], c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'ROI path, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathROI.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(centroidGlobalColXSmooth))
  plt.scatter(centroidGlobalColXSmooth, centroidGlobalRowYSmooth, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(centroidGlobalRowY[0], centroidGlobalColX[0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'Full path smooth, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathFullSmooth.eps')
  plt.show()

  plt.figure()
  c = np.arange(len(particleCentroidROIColXSmooth))
  plt.scatter(particleCentroidROIColXSmooth, particleCentroidROIRowYSmooth, c=c, cmap='jet', edgecolor='none')
  plt.colorbar()
  plt.scatter(particleCentroidROI[0, 1], particleCentroidROI[0, 0], color='red')
  plt.gca().invert_yaxis()
  plt.axis('equal')
  plt.title(f'ROI path smooth, file {saveID}, Particle {particleID}, startX {startColX}, startY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.legend(['Start position', 'Path'])
  plt.savefig(f'{saveBaseName}_PathROISmooth.eps')
  plt.show()

  plt.figure()
  plt.imshow(stack[0], cmap='gray')
  plt.title(f'Frame {SFrame}, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.savefig(f'{saveBaseName}_Frame{SFrame}.eps')
  plt.show()

  plt.figure()
  plt.imshow(rawROI[0, :, :], cmap='gray')
  plt.title(f'Raw ROI, file {saveID}, Particle {particleID}, startX {startColX}, startRowY {startRowY}')
  plt.xlabel('X (pixels)')
  plt.ylabel('Y (pixels)')
  plt.savefig(f'{saveBaseName}_Frame1RawROI.eps')
  plt.show()

  plt.close('all')

DICAnalysis('/content/drive/MyDrive/Colab Notebooks/Copy of Cell1_S1.tif', 'P1_C1_S1_11', 440, 281, 1, 300)